In [ ]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [ ]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

## Analysis
---

### 1a: Daily Distinct Cusips and Total Volume by Retail / Institunional grouping

In [ ]:
def distinctCusips_totalVolume_by_retail_institutional(year_start, year_end):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            RetailThreshold,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM (
            SELECT
                A.TrdExctnDt,
                CASE WHEN A.EntrdVolQt < 100000 THEN 'R' ELSE 'IN' END AS RetailThreshold,
                A.CusipId,
                A.EntrdVolQt
            FROM
                Trace A
            INNER JOIN
                BondIssues B ON A.CusipId = B.CompleteCusip
            INNER JOIN 
                BondIssuers C ON B.IssuerId = C.IssuerId
            WHERE
                A.CntraMpId = 'C' 
                AND C.IndustryGroup <> 4
                AND C.CountryDomicile = 'USA'
                AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
                AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
        ) A
        GROUP BY
            TrdExctnDt,
            RetailThreshold
        ORDER BY
            TrdExctnDt, 
            RetailThreshold
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_by_retail_institutional(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_totalVolume_by_retail_institutional_{}-{}.csv'.format(start, end), index=False)

### 1b: Daily Distinct Cusips and Total Volume

In [ ]:
def distinctCusips_totalVolume(year_start, year_end):
    
    base_query = '''
        SELECT 
            A.TrdExctnDt,
            COUNT(DISTINCT A.CusipId) AS DistinctCusips,
            SUM(A.EntrdVolQt) AS TotalVolume
        FROM
            Trace A
        INNER JOIN
            BondIssues B ON A.CusipId = B.CompleteCusip
        INNER JOIN 
            BondIssuers C ON B.IssuerId = C.IssuerId
        WHERE
            A.CntraMpId = 'C' 
            AND C.IndustryGroup <> 4
            AND C.CountryDomicile = 'USA'
            AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
            AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
        GROUP BY
            TrdExctnDt
        ORDER BY
            TrdExctnDt
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_totalVolume(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_totalVolume_{}-{}.csv'.format(start, end), index=False)

### 2: Daily Total Volume by Side (Buy/Sell) 

In [ ]:
def totalVolume_by_side(year_start, year_end):
    
    base_query = '''
        SELECT 
            A.TrdExctnDt,
            A.RptSideCd,
            SUM(A.EntrdVolQt) as TotalVolume
        FROM
            Trace A
        INNER JOIN
            BondIssues B ON A.CusipId = B.CompleteCusip
        INNER JOIN 
            BondIssuers C ON B.IssuerId = C.IssuerId
        WHERE
            A.CntraMpId = 'C' 
            AND C.IndustryGroup <> 4
            AND C.CountryDomicile = 'USA'
            AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
            AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
        GROUP BY
            TrdExctnDt, 
            RptSideCd
        ORDER BY
            TrdExctnDt, 
            RptSideCd
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = totalVolume_by_side(year_start=start, year_end=end)
df.to_csv('data/output/totalVolume_by_side_{}-{}.csv'.format(start, end), index=False)

### 3: Industry analysis on Cusips and Issuers

In [ ]:
def industry_analysis_on_cusips_issuers(year_start, year_end):
    
    base_query = '''
        SELECT
            IndustryCode,
            DistinctCusips,
            DistinctIssuers
        FROM (
            SELECT 
                C.IndustryCode,
                COUNT(DISTINCT A.CusipId) AS DistinctCusips,
                COUNT(DISTINCT C.IssuerID) AS DistinctIssuers
            FROM
                Trace A
            INNER JOIN
                BondIssues B ON A.CusipId = B.CompleteCusip
            INNER JOIN 
                BondIssuers C ON B.IssuerId = C.IssuerId
            WHERE
                A.CntraMpId = 'C' 
                AND C.IndustryGroup <> 4
                AND C.CountryDomicile = 'USA'
                AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
                AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
            GROUP BY
                C.IndustryCode 
        ) A
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = industry_analysis_on_cusips_issuers(year_start=start, year_end=end)
df.to_csv('data/output/industry_analysis_on_cusips_issuers_{}-{}.csv'.format(start, end), index=False)

### 4: Daily Distinct Cusips and Total Volume by Issuer

In [ ]:
def distinctCusips_totalVolume_by_issuer(year_start, year_end):
    
    base_query = '''
        SELECT
            A.TrdExctnDt,
            C.IssuerId,
            COUNT(DISTINCT A.CusipId) AS DistinctCusips,
            SUM(A.EntrdVolQt) AS TotalVolume
        FROM 
            Trace A
        INNER JOIN
            BondIssues B ON A.CusipId = B.CompleteCusip
        INNER JOIN 
            BondIssuers C ON B.IssuerId = C.IssuerId
        WHERE
            A.CntraMpId = 'C'
            AND C.IndustryGroup <> 4
            AND C.CountryDomicile = 'USA'
            AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
            AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
        GROUP BY
            A.TrdExctnDt, 
            C.IssuerId
        ORDER BY
            A.TrdExctnDt, 
            C.IssuerId
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_by_issuer(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_totalVolume_by_issuer_{}-{}.csv'.format(start, end), index=False)

### 5: Daily Distinct Cusips by Rating

In [ ]:
def distinctCusips_by_rating(year_start, year_end):
    
    base_query = '''
        SELECT
            TrdExctnDt,
            MinimumRating,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                A.CusipId,
                A.TrdExctnDt,
                MIN(B.RatingCategory) AS MinimumRating
            FROM (
                SELECT
                    A.CusipId, 
                    A.TrdExctnDt, 
                    MAX(D.RatingDate) AS MaxRatingDate
                FROM 
                    Trace A
                INNER JOIN
                    BondIssues B ON A.CusipId = B.CompleteCusip
                INNER JOIN 
                    BondIssuers C ON B.IssuerId = C.IssuerId
                LEFT JOIN 
                    BondRatings D ON A.CusipId = D.CompleteCusip 
                    AND D.RatingDate <= A.TrdExctnDt AND RatingCategory IS NOT NULL
                WHERE
                    A.CntraMpId = 'C'
                    AND C.IndustryGroup <> 4
                    AND C.CountryDomicile = 'USA'
                    AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
                    AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
                GROUP BY
                    A.CusipId,
                    A.TrdExctnDt
            ) A
            INNER JOIN 
                BondRatings B ON B.CompleteCusip = A.CusipId AND RatingDate = MaxRatingDate
            GROUP BY
                A.CusipId,
                A.TrdExctnDt
        ) B
        GROUP BY
            TrdExctnDt,
            MinimumRating
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_by_rating(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_by_rating_{}-{}.csv'.format(start, end), index=False)

### 6: Distinct Cusips per Maturity Band

In [ ]:
def distinctCusips_by_maturity(year_start, year_end):
    
    base_query = '''
        SELECT
            MaturityBand,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CASE 
                    WHEN ABS(DATEDIFF(DAY, B.Maturity, B.OfferingDate)) * 1.0 / 360 < 5 THEN 1
                    WHEN ABS(DATEDIFF(DAY, B.Maturity, B.OfferingDate)) * 1.0 / 360 < 15 THEN 2
                    ELSE 3
                END AS MaturityBand,
                A.CusipId
            FROM
                Trace A
            INNER JOIN
                BondIssues B ON A.CusipId = B.CompleteCusip
            INNER JOIN 
                BondIssuers C ON B.IssuerId = C.IssuerId
            WHERE
                A.CntraMpId = 'C' 
                AND C.IndustryGroup <> 4
                AND C.CountryDomicile = 'USA'
                AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
                AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
        ) A
        GROUP BY
            MaturityBand
        ORDER BY
            MaturityBand
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_by_maturity(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_by_maturity_{}-{}.csv'.format(start, end), index=False)

### 7: Distinct Cusips per Investment Grade

In [ ]:
def distinctCusips_by_investmentGrade(year_start, year_end):
    
    base_query = '''
        SELECT 
            InvestmentGrade,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CusipId,
                CASE
                    WHEN RatingCategory < 11 THEN 'Y'
                    WHEN RatingCategory < 25 THEN 'N'
                    ELSE 'NR'
                END AS InvestmentGrade
            FROM (
                SELECT
                    A.CusipId,
                    MIN(D.RatingCategory) AS RatingCategory
                FROM (
                    SELECT
                        A.CusipId,
                        MIN(A.TrdExctnDt) AS FirstTradeExecutionDate
                    FROM 
                        Trace A
                    INNER JOIN
                        BondIssues B ON A.CusipId = B.CompleteCusip
                    INNER JOIN 
                        BondIssuers C ON B.IssuerId = C.IssuerId
                    WHERE
                        A.CntraMpId = 'C'
                        AND C.IndustryGroup <> 4
                        AND C.CountryDomicile = 'USA'
                        AND C.IndustryCode NOT IN (40, 41, 42, 43, 44, 45)
                        AND A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
                    GROUP BY
                        A.CusipId
                ) A
                LEFT JOIN 
                    BondRatings D ON A.CusipId = D.CompleteCusip 
                    AND D.RatingDate <= A.FirstTradeExecutionDate AND RatingCategory IS NOT NULL
                GROUP BY
                    A.CusipId
            ) B
        ) C
        GROUP BY
            InvestmentGrade
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = distinctCusips_by_investmentGrade(year_start=start, year_end=end)
df.to_csv('data/output/distinctCusips_by_investmentGrade_{}-{}.csv'.format(start, end), index=False)

### 8: Distinct Trading Days

In [ ]:
def distinctTradingDays(year_start, year_end):
    
    base_query = '''
        SELECT
            Datadate,
            MktRf, Smb, Hml, Rmw, Cma, Rf, Rm, 
            ABS(Rm) AS AbsoluteRm,
            POWER(Rm, 2) AS SquaredRm, 
            Sum / Count AS Measure
        FROM (
            SELECT
                DataDate, 
                MktRf, Smb, Hml, Rmw, Cma, Rf, Rm,
                ABS(SUM(DReturn) - Rm) AS Sum,
                COUNT(DISTINCT Cusip) AS Count
            FROM (
                SELECT
                    A.Cusip,
                    A.DataDate,
                    LOG(A.PrcCd, EXP(1)) - LOG(LAG(A.PrcCd) OVER (
                        PARTITION BY A.Cusip ORDER BY A.DataDate), EXP(1)
                    ) AS DReturn,
                    B.*
                FROM
                    [dbo].[CrspcSecuritiesDaily] A
                INNER JOIN
                    [dbo].[CrspcFactors] B ON A.DataDate = B.Date
                WHERE
                    B.Date >= '{}-01-1' AND B.Date < '{}-01-01'
            ) A
            GROUP BY
                DataDate,
                MktRf, Smb, Hml, Rmw, Cma, Rf, Rm
        ) B
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    factors.Rm.quantile(0.05)
    df['LeftTail'] = df[]
    
    return df

In [ ]:
start, end = 2002, 2022 # MIN = 2002-07-01 | MAX = 2022-09-30

df = distinctTradingDays(year_start=start, year_end=end)
df.to_csv('data/output/distinctTradingDays_{}-{}.csv'.format(start, end), index=False)

## Herding
---

### 1. Stock Market

In [ ]:
def herding_stockMarket(year_start, year_end):
    
    base_query = '''
    
        DECLARE @LeftTail FLOAT
        SET @LeftTail = (
            SELECT DISTINCT PERCENTILE_CONT(0.05) WITHIN GROUP(ORDER BY Rm) OVER () FROM [dbo].[CrspcFactors]
        )

        DECLARE @RightTail FLOAT
        SET @RightTail = (
            SELECT DISTINCT PERCENTILE_CONT(0.95) WITHIN GROUP(ORDER BY Rm) OVER () FROM [dbo].[CrspcFactors]
        )

        SELECT
            Datadate,
            MktRf, Smb, Hml, Rmw, Cma, Rf, Rm, 
            ABS(Rm) AS AbsoluteRm,
            POWER(Rm, 2) AS SquaredRm, 
            Sum / Count AS Measure,
            CASE 
                WHEN Rm <= @LeftTail THEN 1
                ELSE 0
            END AS LeftTail,
            CASE 
                WHEN Rm >= @RightTail THEN 1
                ELSE 0
            END AS RightTail
        FROM (
            SELECT
                DataDate, 
                MktRf, Smb, Hml, Rmw, Cma, Rf, Rm,
                ABS(SUM(DReturn) - Rm) AS Sum,
                COUNT(DISTINCT Cusip) AS Count
            FROM (
                SELECT
                    A.Cusip,
                    A.DataDate,
                    B.*,
                    LOG(PrcCd, EXP(1)) - LOG(LAG(A.PrcCd) OVER (PARTITION BY A.Cusip ORDER BY A.DataDate), EXP(1)) AS DReturn
                FROM
                    [dbo].[CrspcSecuritiesDaily] A
                INNER JOIN
                    [dbo].[CrspcFactors] B ON A.DataDate = B.Date
                WHERE
                    PrcCd <> 0
                    AND B.Date >= '{}-01-1' AND B.Date < '{}-01-01'
            ) A
            GROUP BY
                DataDate,
                MktRf, Smb, Hml, Rmw, Cma, Rf, Rm
        ) B
        ORDER BY
            DataDate
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = herding_stockMarket(year_start=start, year_end=end)
df.to_csv('data/output/herding_stockMarket_{}-{}.csv'.format(start, end), index=False)

### 2. Auxiliary Measures

In [ ]:
def herding_auxDistinctCusips(year_start, year_end):
    
    base_query = '''
        SELECT
            COUNT(DISTINCT Cusip)
        FROM
            [dbo].[CrspcSecuritiesDaily]
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = herding_auxDistinctCusips(year_start=start, year_end=end)
df.to_csv('data/output/herding_auxDistinctCusips_{}-{}.csv'.format(start, end), index=False)

In [ ]:
def herding_auxDistinctCusips_by_exchange(year_start, year_end):
    
    base_query = '''
        SELECT
            Exchange,
            COUNT(DISTINCT Cusip)
        FROM
            [dbo].[CrspcSecuritiesDaily]
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
        GROUP BY
            Exchange
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = herding_auxDistinctCusips_by_exchange(year_start=start, year_end=end)
df.to_csv('data/output/herding_auxDistinctCusips_by_exchange_{}-{}.csv'.format(start, end), index=False)

In [ ]:
def herding_auxDistinctCusips_by_industry(year_start, year_end):
    
    base_query = '''
        SELECT
            Industry,
            COUNT(DISTINCT Cusip)
        FROM
            [dbo].[CrspcSecuritiesDaily]
        WHERE
            DataDate >= '{}-01-1' AND DataDate < '{}-01-01'
        GROUP BY
            Industry
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [ ]:
start, end = 2002, 2022

df = herding_auxDistinctCusips_by_industry(year_start=start, year_end=end)
df.to_csv('data/output/herding_auxDistinctCusips_by_industry_{}-{}.csv'.format(start, end), index=False)